In [1]:
!pip install transformers[torch] datasets nltk

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.0 MB/s eta 0:00:00


In [2]:
import json
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import re
from datasets import Dataset
from sklearn.metrics import f1_score
from transformers import (
    TrainingArguments,
    Trainer,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    EarlyStoppingCallback,
    MarianMTModel,
    MarianTokenizer
)
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MultiLabelBinarizer
from nltk.corpus import wordnet
import nltk
from nltk.tokenize import word_tokenize
import random
import re

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
classes = ['CE', 'ENV', 'BME', 'PE', 'METAL', 'ME', 'EE', 'CPE', 'OPTIC', 'NANO', 'CHE',
           'MATENG', 'AGRI', 'EDU', 'IE', 'SAFETY', 'MATH', 'MATSCI']

mlb = MultiLabelBinarizer(classes=classes)

In [5]:
with open('data/train_for_student.json', 'r', encoding='utf-8') as f:
    train_for_student = json.load(f)


In [6]:
# You can choose different language pairs. Here we use English to German and back.
en_de_model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-de").to(device)
en_de_tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-de")

de_en_model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-de-en").to(device)
de_en_tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-de-en")

def translate(texts, model, tokenizer, language=""):
    formatted_text = [">>{}<< {}".format(language, text) for text in texts]
    tokens = tokenizer(formatted_text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    translated = model.generate(**tokens)
    translated_texts = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return translated_texts

def backtranslate(texts, source_lang="en", target_lang="de"):
    if target_lang == "de":
        translations = translate(texts, en_de_model, en_de_tokenizer, language=target_lang)
    else:
        translations = translate(texts, de_en_model, de_en_tokenizer, language=target_lang)

    if source_lang == "en":
        back_translations = translate(translations, de_en_model, de_en_tokenizer, language=source_lang)
    else:
        back_translations = translate(translations, en_de_model, en_de_tokenizer, language=source_lang)

    return back_translations


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

In [7]:
# Improved text preprocessing
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s\.\-\,]', '', text)
    return text.lower().strip()

# Enhanced data augmentation techniques
def synonym_replacement(words, n):
    new_words = words.copy()
    random_word_list = list(set([word for word in words if word.isalnum()]))
    random.shuffle(random_word_list)
    num_replaced = 0
    for random_word in random_word_list:
        synonyms = []
        for syn in wordnet.synsets(random_word):
            for l in syn.lemmas():
                synonyms.append(l.name())
        if len(synonyms) >= 1:
            synonym = random.choice(list(set(synonyms)))
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= n:
            break
    return new_words

def random_insertion(words, n):
    new_words = words.copy()
    for _ in range(n):
        add_word(new_words)
    return new_words

def add_word(new_words):
    synonyms = []
    counter = 0
    while len(synonyms) < 1:
        random_word = new_words[random.randint(0, len(new_words)-1)]
        synonyms = get_synonyms(random_word)
        counter += 1
        if counter >= 10:
            return
    random_synonym = synonyms[0]
    random_idx = random.randint(0, len(new_words)-1)
    new_words.insert(random_idx, random_synonym)

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            synonym = l.name().replace("_", " ").replace("-", " ").lower()
            synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
            synonyms.add(synonym)
    if word in synonyms:
        synonyms.remove(word)
    return list(synonyms)

def random_swap(words, n):
    new_words = words.copy()
    for _ in range(n):
        new_words = swap_word(new_words)
    return new_words

def swap_word(new_words):
    random_idx_1 = random.randint(0, len(new_words)-1)
    random_idx_2 = random_idx_1
    counter = 0
    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words)-1)
        counter += 1
        if counter > 3:
            return new_words
    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
    return new_words

def random_deletion(words, p):
    if len(words) == 1:
        return words
    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r > p:
            new_words.append(word)
    if len(new_words) == 0:
        rand_int = random.randint(0, len(words)-1)
        return [words[rand_int]]
    return new_words

def eda(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
    words = word_tokenize(sentence)
    words = [word for word in words if word is not None]
    num_words = len(words)

    augmented_sentences = []
    num_new_per_technique = int(num_aug/4)+1

    #sr
    for _ in range(num_new_per_technique):
        n_sr = max(1, int(alpha_sr*num_words))
        a_words = synonym_replacement(words, n_sr)
        augmented_sentences.append(' '.join(a_words))

    #ri
    for _ in range(num_new_per_technique):
        n_ri = max(1, int(alpha_ri*num_words))
        a_words = random_insertion(words, n_ri)
        augmented_sentences.append(' '.join(a_words))

    #rs
    for _ in range(num_new_per_technique):
        n_rs = max(1, int(alpha_rs*num_words))
        a_words = random_swap(words, n_rs)
        augmented_sentences.append(' '.join(a_words))

    #rd
    for _ in range(num_new_per_technique):
        a_words = random_deletion(words, p_rd)
        augmented_sentences.append(' '.join(a_words))

    augmented_sentences = [sentence for sentence in augmented_sentences if len(sentence) > 10]
    random.shuffle(augmented_sentences)

    if num_aug >= 1:
        augmented_sentences = augmented_sentences[:num_aug]
    else:
        keep_prob = num_aug / len(augmented_sentences)
        augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

    augmented_sentences.append(sentence)
    return augmented_sentences

def augment_text(text, title_length, n_aug=9):
    title = text[:title_length]
    body = text[title_length:]
    augmented_bodies = eda(body, num_aug=n_aug)
    return [f"{title}{aug_body}" for aug_body in augmented_bodies]

def process_data(data, augment=True, batch_size=128):
    rows = []
    all_classes = []

    # Process data in batches
    for i in range(0, len(data), batch_size):
        batch = list(data.items())[i:i+batch_size]

        texts = []
        for id, info in batch:
            title = clean_text(info['Title'])
            abstract = clean_text(info['Abstract'])
            text = f"{title} [SEP] {abstract}"
            texts.append(text)

        # Perform backtranslation on the batch
        backtranslated_texts = backtranslate(texts)

        for j, (id, info) in enumerate(batch):
            original_text = texts[j]
            backtranslated_text = backtranslated_texts[j]
            title_length = len(clean_text(info['Title'])) + 6

            if augment and 'Classes' in info:
                # Add original text
                rows.append({"text": original_text})
                all_classes.append(info['Classes'])

                # Add backtranslated text
                rows.append({"text": backtranslated_text})
                all_classes.append(info['Classes'])

                # Apply EDA on both original and backtranslated texts
                original_augmented = augment_text(original_text, title_length, n_aug=9)
                backtranslated_augmented = augment_text(backtranslated_text, title_length, n_aug=4)

                for aug_text in original_augmented + backtranslated_augmented:
                    rows.append({"text": aug_text})
                    all_classes.append(info['Classes'])
            else:
                rows.append({"text": original_text})
                if 'Classes' in info:
                    all_classes.append(info['Classes'])

    df = pd.DataFrame(rows)
    if all_classes:
        labels = mlb.fit_transform(all_classes)
        df['labels'] = [label.tolist() for label in labels]
    return df


In [8]:
train_df = process_data(train_for_student)

In [67]:
print(train_df.shape[0])
train_df.head(5)

7718


,text,labels
0,activated carbon derived from bacterial cellul...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, ..."
1,The activated carbon obtained from bacterial c...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, ..."
2,activated carbon derived from bacterial cellul...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, ..."
3,activated carbon derived from bacterial cellul...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, ..."
4,activated carbon derived from bacterial cellul...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, ..."


In [68]:
train_data, test_data = train_test_split(train_df, test_size=0.4, random_state=42)
val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)


hg_train_data = Dataset.from_pandas(train_data)
hg_val_data = Dataset.from_pandas(val_data)
hg_test_data = Dataset.from_pandas(test_data)

print(hg_train_data)

Dataset({
    features: ['text', 'labels', '__index_level_0__'],
    num_rows: 4630
})


In [69]:
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=True)

def tokenize_dataset(data):
    tokenized_data = tokenizer(
        data['text'],
        truncation=True,
        padding='max_length',
        max_length=512,
    )
    return tokenized_data

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [70]:
dataset_train = hg_train_data.map(tokenize_dataset, batched=True)
dataset_val = hg_val_data.map(tokenize_dataset, batched=True)
dataset_test = hg_test_data.map(tokenize_dataset, batched=True)

print(dataset_train[0])

Map:   0%|          | 0/4630 [00:00<?, ? examples/s]

Map:   0%|          | 0/1544 [00:00<?, ? examples/s]

Map:   0%|          | 0/1544 [00:00<?, ? examples/s]

{'text': 'a durable rechargeable zinc-air battery via self-supported mnox-s air electrode [SEP]2021 the authorsthe durability and performance during an air electrode bi-electrode rechargeable a crucial performance on mvdec zinc-air batteries zabs . a typical air electrode fabricated using conductive carbon and a polymeric sulfur-doped suffers rapid degradation of charging , stemming from charge bubbles erosion and carbon corrosion . this work presents well durable air electrode having a self-supported binder manganese oxides mnox-s electrocatalyst prepared through hydrothermal process the 74 ambient for sulfurization . upon sulfurization , mno2 nanoflakes are transformed into mnox-s having a heterostructure . mn3o4 discharge mno2 and mnsx of density provided superior electrochemical properties for both oxygen mnox-s reaction tafel slope of 68 higher and oxygen evolution reaction tafel slope of 80 mvdec macm2 observed electrodes using mnox-s and mno2 electrocatalysts were scrutinized fo

In [71]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(classes),
    problem_type='multi_label_classification'
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [72]:
# Optimizing thresholds for each label
def optimize_thresholds(predictions, labels, num_points=100):
    best_thresholds = []

    for i in range(predictions.shape[1]):
        best_f1 = 0
        best_threshold = 0.5

        # Get predictions for current class
        class_predictions = predictions[:, i]
        sorted_preds = np.sort(class_predictions)

        # Try different threshold values
        for percentile in range(1, num_points):
            threshold = sorted_preds[int(len(sorted_preds) * (percentile/num_points))]
            binary_predictions = (class_predictions >= threshold).astype(int)
            f1 = f1_score(labels[:, i], binary_predictions)

            if f1 > best_f1:
                best_f1 = f1
                best_threshold = threshold

        best_thresholds.append(best_threshold)

    return best_thresholds

# Custom compute_metrics function
def compute_metrics(p):
    logits = p.predictions
    labels = p.label_ids
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.tensor(logits))

    # Optimize thresholds
    thresholds = optimize_thresholds(probs.numpy(), labels)
    predictions = np.array([(probs[:, i] > thresholds[i]).numpy().astype(int) for i in range(len(thresholds))]).T

    # Calculate F1 score
    f1 = f1_score(y_true=labels, y_pred=predictions, average='macro')
    return {
        'macro_f1': f1,
        'thresholds': thresholds
    }


In [73]:
class MultilabelTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.label_weights = self.compute_class_weights()

    def compute_class_weights(self):
        labels = np.array(self.train_dataset['labels'])
        pos_counts = np.sum(labels, axis=0)
        neg_counts = len(labels) - pos_counts
        weights = np.sqrt(neg_counts / (pos_counts + 1))  # Square root smoothing
        return torch.FloatTensor(weights).to(self.args.device)

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        # Improved focal loss with dynamic alpha
        alpha = 0.75  # Increased focus on positive samples
        gamma = 2.0

        bce_loss = torch.nn.BCEWithLogitsLoss(reduction='none',
                                             pos_weight=self.label_weights)
        base_loss = bce_loss(logits, labels.float())

        probs = torch.sigmoid(logits)
        p_t = probs * labels + (1 - probs) * (1 - labels)
        focal_weight = ((1 - p_t) ** gamma) * alpha

        loss = (focal_weight * base_loss).mean()

        return (loss, outputs) if return_outputs else loss


# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=5e-5,
    num_train_epochs=20,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=2,
    warmup_ratio=0.1,
    weight_decay=0.01,
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model='macro_f1',
    greater_is_better=True,
    fp16=True,
    gradient_checkpointing=True,
    save_total_limit=2,
    seed=42
)

trainer = MultilabelTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    data_collator=DataCollatorWithPadding(tokenizer)
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [74]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss,Macro F1,Thresholds
100,0.112800,0.089639,0.645869,"[0.5093525648117065, 0.5047910809516907, 0.4978561997413635, 0.47245529294013977, 0.5429844260215759, 0.45177996158599854, 0.4671798348426819, 0.47429633140563965, 0.49175333976745605, 0.5054624080657959, 0.5587182641029358, 0.529628336429596, 0.5226286053657532, 0.4733072519302368, 0.4950791895389557, 0.49463289976119995, 0.49201270937919617, 0.514537513256073]"
200,0.049300,0.034455,0.931031,"[0.5805304050445557, 0.5916053056716919, 0.6068480014801025, 0.5565803647041321, 0.544105052947998, 0.5193994641304016, 0.5489152669906616, 0.5281683802604675, 0.49674609303474426, 0.5919591188430786, 0.5204353928565979, 0.5443170070648193, 0.5274686217308044, 0.5583571195602417, 0.4723183512687683, 0.5627171993255615, 0.49847036600112915, 0.5130890607833862]"
300,0.018200,0.012570,0.985513,"[0.43014734983444214, 0.4721510708332062, 0.6585273146629333, 0.4287714958190918, 0.43860483169555664, 0.6177433133125305, 0.4392662048339844, 0.4431183934211731, 0.4602585434913635, 0.7111300826072693, 0.5046309232711792, 0.5939626097679138, 0.43914595246315, 0.5414391756057739, 0.39711982011795044, 0.46386927366256714, 0.41810283064842224, 0.5139963030815125]"
400,0.008700,0.006692,0.989066,"[0.378459095954895, 0.3858373761177063, 0.7446563243865967, 0.36285629868507385, 0.36229208111763, 0.35488009452819824, 0.3565586805343628, 0.35209035873413086, 0.3501991033554077, 0.7133318185806274, 0.509802520275116, 0.7259401679039001, 0.3559987545013428, 0.6594051122665405, 0.3585776388645172, 0.36579644680023193, 0.3839299976825714, 0.6344300508499146]"
500,0.005200,0.004477,0.988931,"[0.31153157353401184, 0.5960217714309692, 0.7750945687294006, 0.2959415912628174, 0.30033501982688904, 0.3284217119216919, 0.2887696325778961, 0.3013620376586914, 0.3214600086212158, 0.7590468525886536, 0.6082450747489929, 0.7685595154762268, 0.3798958659172058, 0.6923300623893738, 0.28756770491600037, 0.3579041063785553, 0.28140562772750854, 0.6834195256233215]"
600,0.003800,0.003826,0.987773,"[0.31289464235305786, 0.5990160703659058, 0.8091251850128174, 0.281109482049942, 0.2845759987831116, 0.36761075258255005, 0.2622745931148529, 0.25552940368652344, 0.4422149062156677, 0.8175744414329529, 0.6167050004005432, 0.8033281564712524, 0.6941990256309509, 0.7384804487228394, 0.27231472730636597, 0.7200731039047241, 0.2762022018432617, 0.7362110018730164]"
700,0.003300,0.003132,0.987954,"[0.2728956639766693, 0.38294878602027893, 0.8279673457145691, 0.2544163167476654, 0.280419260263443, 0.2958398759365082, 0.7953404188156128, 0.32018324732780457, 0.2820972800254822, 0.8162599205970764, 0.6588566303253174, 0.79739910364151, 0.30259713530540466, 0.7275885343551636, 0.2735743820667267, 0.30954527854919434, 0.2929997146129608, 0.6946134567260742]"


Trainer is attempting to log a value of "[0.5093525648117065, 0.5047910809516907, 0.4978561997413635, 0.47245529294013977, 0.5429844260215759, 0.45177996158599854, 0.4671798348426819, 0.47429633140563965, 0.49175333976745605, 0.5054624080657959, 0.5587182641029358, 0.529628336429596, 0.5226286053657532, 0.4733072519302368, 0.4950791895389557, 0.49463289976119995, 0.49201270937919617, 0.514537513256073]" of type <class 'list'> for key "eval/thresholds" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two v

TrainOutput(global_step=700, training_loss=0.034918776558978216, metrics={'train_runtime': 309.3848, 'train_samples_per_second': 299.304, 'train_steps_per_second': 4.654, 'total_flos': 1.176538522871808e+16, 'train_loss': 0.034918776558978216, 'epoch': 9.655172413793103})

In [79]:
optimized_thresholds = [0.40229520928454536, 0.5046116087702355, 0.6885821342468262, 0.3796615387598403, 0.3935696326415061, 0.4158164678568413, 0.45118921086986325, 0.3820061660194184, 0.40636485712528843, 0.7021092632450649, 0.5681991151400977, 0.6804385459036598, 0.4609905643449374, 0.6272725296908814, 0.3652302434284041, 0.4677710297409992, 0.37180246905461555, 0.6271852731704426]
test_predictions = trainer.predict(dataset_test)

test_predictions

PredictionOutput(predictions=array([[-1.2714844, -2.5332031, -2.4355469, ..., -2.2636719, -1.7441406,
        -1.5400391],
       [-1.8515625,  1.8417969, -2.1464844, ..., -1.9589844, -1.4228516,
        -1.0810547],
       [-1.0595703, -2.9355469, -1.9931641, ..., -1.8417969, -1.2041016,
         1.4472656],
       ...,
       [-1.0625   ,  1.5908203, -1.7011719, ..., -1.1464844, -0.9169922,
        -1.3876953],
       [-2.0546875, -1.5214844,  1.7568359, ..., -1.8271484, -1.7470703,
        -1.1904297],
       [-1.40625  , -1.4003906, -2.1289062, ..., -2.0195312, -2.296875 ,
        -1.2958984]], dtype=float32), label_ids=array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), metrics={'test_loss': 0.006197392009198666, 'test_macro_f1': 0.9868014362923709, 'test_thresholds': [0.3652302324771881, 0.7364006042480469, 0.7270074486732483, 0

In [80]:
# Apply sigmoid to the logits to get probabilities
test_probabilities = torch.sigmoid(torch.tensor(test_predictions.predictions)).numpy()

# Apply the thresholds to get binary predictions
binary_predictions = np.zeros(test_probabilities.shape)
for i in range(len(classes)):  # Iterate over the classes
    binary_predictions[:, i] = (test_probabilities[:, i] >= optimized_thresholds[i]).astype(int)

# Extract the true labels
true_labels = test_predictions.label_ids

In [81]:
# Trainer evaluate
trainer.evaluate(dataset_test)

Trainer is attempting to log a value of "[0.3652302324771881, 0.7364006042480469, 0.7270074486732483, 0.7111300826072693, 0.423517644405365, 0.3260568082332611, 0.6834195256233215, 0.6529056429862976, 0.717507004737854, 0.38358360528945923, 0.5775549411773682, 0.7302899360656738, 0.7279754281044006, 0.39291906356811523, 0.3744474947452545, 0.3270232081413269, 0.4684866666793823, 0.3729618489742279]" of type <class 'list'> for key "eval/thresholds" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.006197392009198666,
 'eval_macro_f1': 0.9868014362923709,
 'eval_thresholds': [0.3652302324771881,
  0.7364006042480469,
  0.7270074486732483,
  0.7111300826072693,
  0.423517644405365,
  0.3260568082332611,
  0.6834195256233215,
  0.6529056429862976,
  0.717507004737854,
  0.38358360528945923,
  0.5775549411773682,
  0.7302899360656738,
  0.7279754281044006,
  0.39291906356811523,
  0.3744474947452545,
  0.3270232081413269,
  0.4684866666793823,
  0.3729618489742279],
 'eval_runtime': 6.9728,
 'eval_samples_per_second': 221.431,
 'eval_steps_per_second': 7.027,
 'epoch': 9.655172413793103}

In [82]:
# Compute f1 metric
final_f1 = f1_score(true_labels, binary_predictions, average='macro')

print("\nFinal Evaluation:")
print(f"Macro F1 Score: {final_f1}")


Final Evaluation:
Macro F1 Score: 0.9948772686790323


In [83]:
with open('data/test_for_student.json', 'r', encoding='utf-8') as f:
    test_for_student = json.load(f)

In [84]:
def process_test_data(data):
    texts = []
    ids = []
    for id, info in data.items():
        title = clean_text(info['Title'])
        abstract = clean_text(info['Abstract'])
        text = f"{title} [SEP] {abstract}"
        texts.append(text)
        ids.append(id)
    return texts, ids

In [48]:
test_texts, test_ids = process_test_data(test_for_student)

In [85]:
# Tokenize the test data
inputs = tokenizer(test_texts, padding=True, truncation=True, return_tensors="pt")
inputs = {key: value.to(device) for key, value in inputs.items()}  # Move inputs to the same devic

In [86]:
# Make predictions using the model
model.to(device)
model.eval()
with torch.no_grad():
    logits = model(**inputs).logits

# Convert logits to probabilities using sigmoid function
sigmoid = torch.nn.Sigmoid()
test_probabilities = sigmoid(logits).cpu().numpy()

In [87]:
# Create a binary predictions array based on thresholds
binary_predictions = np.zeros(test_probabilities.shape)
for i in range(len(classes)):  # Iterate over each class
    binary_predictions[:, i] = (test_probabilities[:, i] >= optimized_thresholds[i]).astype(int)

In [88]:
output_predictions = []
for i, test_id in enumerate(test_ids):
    row = [test_id] + binary_predictions[i].tolist()
    output_predictions.append(row)

# Convert the output to a DataFrame for easy saving
columns = ["id"] + classes
submission_df = pd.DataFrame(output_predictions, columns=columns)

# Save to a CSV file (optional)
submission_df.to_csv("submission.csv", index=False)
print("Predictions saved to submission.csv")

Predictions saved to submission.csv


In [92]:
# Save tokenizer
tokenizer.save_pretrained('./research_classification_bert')

# Save model
trainer.save_model('./research_classification_bert')

!zip -r research_classification_bert.zip research_classification_bert/

  adding: research_classification_bert/ (stored 0%)
  adding: research_classification_bert/model.safetensors (deflated 7%)
  adding: research_classification_bert/vocab.txt (deflated 53%)
  adding: research_classification_bert/training_args.bin (deflated 51%)
  adding: research_classification_bert/tokenizer.json (deflated 71%)
  adding: research_classification_bert/special_tokens_map.json (deflated 42%)
  adding: research_classification_bert/tokenizer_config.json (deflated 76%)
  adding: research_classification_bert/config.json (deflated 61%)
